Bollinger Bands&copy; are used in trend following strategies.  See https://www.bollingerbands.com for more information.

We use a moving average of the price of a security, and an upper and lower band around this price.  When the price moves above the upper band, we buy and when the price moves below the lower band we sell.

First, lets load some data.  We are going to use a subset of bitcoin 1 minute bars downloaded from [Kaggle](https://www.kaggle.com/mczielinski/bitcoin-historical-data)

For the purposes of this tutorial, we will trade in 5 minute bars instead of 1 minute bars, so lets resample the OHLCV data to 5 minutes


In [1]:
import sys
sys.path.insert(0, '/Users/sal/Developer/pyqstrat')

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed") # ignore pesky warning, see https://github.com/numpy/numpy/pull/432

import math
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import os

import pyqstrat as pq

pq.set_defaults() # Set some display defaults to make dataframes and plots easier to look at

try:
    file_path = os.path.dirname(os.path.realpath(__file__)) + '/../examples/bitcoin_1min.csv' # If we are running from unit tests
except:
    file_path = '../examples/bitcoin_1min.csv'
    
prices = pd.read_csv(file_path)

dates = pd.to_datetime(prices.date).values
o = prices.o.values
h = prices.h.values
l = prices.l.values
c = prices.c.values
v = prices['v.usd'].values

Lets create a market data object from this data and plot the first day of the bars to see how they look.

Note that the MarketData constructor **takes c as its second argument, not o**.  This is because h, l, c and v are optional, but c is not.

In [ ]:
md = pq.MarketData(dates, c, o, h, l, v)
md.resample(sampling_frequency='5 min', inplace = True)
md.plot(date_range = ('2018-03-24 09:30', '2018-03-24 16:00'))
md.describe(time_distribution_frequency = '30 mins')

Inferred Frequency: 5.0 minutes


,date,o,h,l,c,v
count,1001,1001,1001,1001,1001,1001
num_missing,0,0,0,0,0,0
pct_missing,0,0,0,0,0,0
min,2018-03-23 12:40:00,7873,7889,7831,7878,1425
max,2018-03-27 00:00:00,9006,9020,8970,9006,6.155e+06


Errors:
Warnings:


,v,l,h,o,c,ret,warning
date,,,,,,,
2018-03-26 22:05:00,1.771e+06,7908,8102,7915,8102,nan,o ret > 10 std: 0.001931
2018-03-26 22:10:00,1.361e+06,8046,8150,8103,8150,0.0237,o ret > 10 std: 0.001931
2018-03-26 22:00:00,9.338e+04,7876,7925,7888,7908,nan,h ret > 10 std: 0.001774
2018-03-26 22:05:00,1.771e+06,7908,8102,7915,8102,0.02233,h ret > 10 std: 0.001774
2018-03-26 22:00:00,9.338e+04,7876,7925,7888,7908,nan,c ret > 10 std: 0.001952
2018-03-26 22:05:00,1.771e+06,7908,8102,7915,8102,0.02453,c ret > 10 std: 0.001952


Time distribution:


<Figure size 432x288 with 0 Axes>

This looks reasonable.  The describe() function allows us to do some checks on the market data to make sure its ready for use in a strategy.  It tells us:

1.  How much data is missing, and the most common bar frequency.

2.  Errors - Negative values, highs or lows that are lower or higher than opens or closes.

3.  Warnings - Returns for o, h, l, c, v that are more than n (default = 10) standard deviations off the mean.

5.  Time distribution of price frequency and trade volume - This tells you, for example, if the market data includes after-market prices or not.


The next step is to add the Bollinger Bands.  Lets set the middle band as the 20 bar simple moving average of the close, and the upper and lower bands to be 1 standard deviation away from the middle band. This time, instead of plotting candles, we will plot the close price to display a cleaner graph.

In [ ]:
def sma(marketdata, lookback_period): # simple moving average
    return pd.Series(marketdata.c).rolling(window = lookback_period).mean().values

def band(marketdata, lookback_period, num_std, upper):
    std = pd.Series(marketdata.c).rolling(window = 20).std()
    return sma(marketdata, lookback_period = lookback_period) + num_std * std * (1 if upper else -1)

num_std = 1 # Each band is 2 standard deviations away from the moving average of price

lookback_period = 20 # Length of moving average window
mid_band = sma(md, lookback_period)
upper_band = band(md, lookback_period, num_std, upper = True)
lower_band = band(md, lookback_period, num_std, upper = False)

indicator_subplot = pq.Subplot([pq.TimeSeries('price', md.dates, md.c, color = 'blue'), 
                     pq.TimeSeries('sma', md.dates, mid_band, line_type = 'dotted', color = 'red'),
                     pq.TimeSeries('upper band', md.dates, upper_band, line_type = 'dashed', color = 'green'),
                     pq.TimeSeries('lower_band', md.dates, lower_band, line_type = 'dashed', color = 'green')],
                     ylabel = 'Indicators')
plot = pq.Plot(indicator_subplot, title = 'BTC')
plot.draw()

The bands and the moving average are called "indicators" in pyqstrat.  Indicators are the fundamental building blocks of any strategy.

Now lets create a signal.  A signal uses marketdata and indicators and tells a strategy when to create orders.  In this case we buy when the price crosses the upper band and exit when the price crosses the moving average.  Similarly, we sell when the price crosses the lower band and exit when it crosses the moving average.  We create 4 different values of the signal, +2, -2, +1, -1 to signal all 4 of these conditions.

In [ ]:
def bollinger_band_signal(md, ind): # md is MarketData, ind is indicator values, a dictionary of indicator values
    signal = np.where(md.h > np.nan_to_num(ind['upper_band']), 2, 0)
    signal = np.where(md.l < np.nan_to_num(ind['lower_band']), -2, signal)
    signal = np.where((md.h > np.nan_to_num(ind['mid_band'])) & (signal == 0), 1, signal) # price crossed above simple moving avg but not above upper band
    signal = np.where((md.l < np.nan_to_num(ind['mid_band'])) & (signal == 0), -1, signal) # price crossed below simple moving avg but not below lower band
    return signal

Let's plot the signal in a subplot along with the indicators and check whether it makes sense

In [ ]:
signal = bollinger_band_signal(md, {'upper_band' : upper_band, 'lower_band' : lower_band, 'mid_band' : mid_band})
signal_subplot = pq.Subplot([pq.TimeSeries('signal', md.dates, signal)], ylabel = 'Signal')
plot = pq.Plot([indicator_subplot, signal_subplot], title = 'BTC', show_date_gaps = False)
plot.draw()

This is hard to see clearly so let's look at a smaller date range

In [ ]:
plot = pq.Plot([indicator_subplot, signal_subplot], title = 'BTC', date_range = ('2018-03-24', '2018-03-25'))
plot.draw()

This looks good.  The signal behaves as we would expect.

The next step is to create a trading rule.  pyqstrat will call a trading rule function when the value of it's associated signal changes value, or when we have live orders in the market that have not been filled.  A trading rule decides whether or not to take action on the signal.  If it decides to take action, it returns a list of one or more orders.  This is where margin decisions are made as well, i.e. the trading rule knows how much margin is needed, and will not place orders if margin is exceeded.

Trading rules are usually path dependent, i.e. they cannot be vectorized like indicators and signals can, since can depend on current position, risk, or other data that changes with time.  

Because of this path dependence, pyqstrat will call order rule functions once per date.  This is the part of the code (along with market simulation) that you should optimize as much as you can.  My advice is to use numpy arrays as much as possible here, avoiding slow parts of pandas code.

Lets build the trading rule.  We will risk 10% of our current equity when going long or short.  When entering we will enter a stop order.  This means the system will only enter if the price crosses the current close during the next bar.  We are trying to avoid a case where we cross a bollinger band in this bar but then price immediately goes the other way in the next bar, in which case we should not enter the trade.  We will ignore margin in this case for simplicity.  If we wanted to check margin, we would add code here that would check if margin would be exceeded (based on broker specific logic), and modify the order qty accordingly.

We are using predefined order reason codes when we create orders below.  Reason codes are optional strings that we use to identify why we entered an order and are useful for debugging and for plotting.  You can use an arbitrary string as a reason code.  If you later want to identify those trades in a plot, you will have to provide a mapping between your reason codes and the markers used for that reason code in a trade plot.

In [ ]:
def bollinger_band_trading_rule(strategy, symbol, i, date, marketdata, indicator_values, signal_values, account):
    curr_pos = account.position(symbol, date)
    signal_value = signal_values[i]
    risk_percent = 0.1
    
    # if we don't already have a position, check if we should enter a trade
    if math.isclose(curr_pos, 0):
        if signal_value == 2 or signal_value == -2:
            curr_equity = account.equity(date)
            order_qty = np.round(curr_equity * risk_percent / marketdata.c[i] * np.sign(signal_value))
            print(f'order qty: {order_qty} curr_equity: {curr_equity} risk_percent: {risk_percent} c: {marketdata.c[i]} sign: {np.sign(signal_value)}')
            trigger_price = marketdata.c[i]
            reason_code = pq.ReasonCode.ENTER_LONG if order_qty > 0 else pq.ReasonCode.ENTER_SHORT
            return [pq.StopLimitOrder(symbol, date, order_qty, trigger_price, reason_code = reason_code)]
    else: # We have a current position, so check if we should exit
        if (curr_pos > 0 and signal_value == -1) or (curr_pos < 0 and signal_value == 1):
            order_qty = -curr_pos
            reason_code = pq.ReasonCode.EXIT_LONG if order_qty < 0 else pq.ReasonCode.EXIT_SHORT
            return [pq.MarketOrder(symbol, date, order_qty, reason_code = reason_code)]
    return []


We need to write one more function, a market simulator.  The market simulator decides whether open orders will be filled, and at what price.  This is where you would add slippage, commissions and fees.  You can add a different market simulator for each symbol, since liquidity and market rules are clearly different when trading an IBM stock on NYSE versus trading a E-mini future on CME.  pyqstrat will call the market simulator on the next bar after you add an order since you obviously cannot get filled using prices you already saw and used to construct your order.  In this simulator we fill buy market orders half way between the open and the high for the current bar, and sells half way between the open and the low. Stop orders are filled halfway between the trigger price and the high or low.  We ignore commissions and fees for simplicity.

In [ ]:
def market_simulator(strategy, orders, i, date, md):
    trades = []
    
    o, h, l, c = md.o[i], md.h[i], md.l[i], md.c[i]
    
    for order in orders:
        trade_price = np.nan
        
        if isinstance(order, pq.MarketOrder):
            trade_price = 0.5 * (o + h) if order.qty > 0 else 0.5 * (o + l)
        elif isinstance(order, pq.StopLimitOrder):
            if (order.qty > 0 and h > order.trigger_price) or (order.qty < 0 and l < order.trigger_price): # A stop order
                trade_price = 0.5 * (order.trigger_price + h) if order.qty > 0 else 0.5 * (order.trigger_price + l)
        else:
            raise Exception(f'unexpected order type: {order}')
            
        if np.isnan(trade_price): continue
            
        trade = pq.Trade(order.symbol, date, order.qty, trade_price, order = order, commission = 0, fee = 0)
        order.status = 'filled'
        print(f'got trade: {trade}')
                           
        trades.append(trade)
                           
    return trades

Now lets create the strategy and run it.

We will instantiate a few objects:


1.  Contract - a symbol represents a real or virtual instrument.  Each contract contains a symbol, marketdata and a multiplier. The multiplier is used in cases where price and quantity are in different units.  For example, for most equity options, a quantity of 1 represents 100 shares, but a price of 0.50 represents 0.50 per share.


2.  Account - an account is tied to a strategy and keeps track of your current equity, realized and unrealized P&L as well as commissions and fees.  An account has a calculation frequency that is usually slower than the strategy.  For example, a strategy may trade with 1 minute bars, but you may only need to calculate P&L at end of day.  However, at some points along the way, you may need intraday equity, etc.  For example, the strategy may need to look at current equity to determine how to size orders.  In these cases, the account will automatically recalculate when you ask it for this data.  An account contains one or more symbols.  In this case we add one symbol, IBM, to the account, and set our starting equity to $1mm.


3.  Strategy - You run a strategy after adding indicators, signals, trading rules, and market simulators to generate orders and trades.  Running a strategy also calculates returns and P&L.


4.  Portfolio - A portfolio is a collection of strategies.  For example, you may have two or more uncorrelated strategies that you want to run concurrently and rebalance periodically.  You can add these to a single portfolio and add rebalance rules to compute returns and P&L.

In [ ]:
contract = pq.Contract('BTC', multiplier = 1., marketdata = md)
strategy = pq.Strategy([contract])

# since pqstrat expects the indicator to take one argument, market data, and our sma function takes 2 arguments, we wrap it in a lambda 
strategy.add_indicator('mid_band', lambda md : sma(md, lookback_period))
strategy.add_indicator('upper_band', lambda md: band(md, lookback_period, num_std, upper = True))
strategy.add_indicator('lower_band', lambda md: band(md, lookback_period, num_std, upper = False))
strategy.add_signal('bb_signal', bollinger_band_signal)

# ask pqstrat to call our trading rule when the signal has one of the values [-2, -1, 1, 2]
strategy.add_rule('bb_trading_rule', bollinger_band_trading_rule, 
                  signal_name = 'bb_signal', sig_true_values = [-2, -1, 1, 2])

strategy.add_market_sim(market_simulator, symbols = ['BTC'])

portfolio = pq.Portfolio()
portfolio.add_strategy('bb_strategy', strategy)
portfolio.run()

Lets plot our strategy and our portfolio and evaluate our returns  

In [ ]:
strategy.plot()

In [ ]:
strategy.df_returns()

In [ ]:
strategy.evaluate_returns();

Note that we calculate P&L once a day so the equity line produced when we evaluate returns has fewer points than the equity line when we plot the strategy itself.

Lets add two to the portfolio and we will run the two strategies together.  We will use two bollinger band strategy with different lookback periods and standard deviations.

In [ ]:
from pyqstrat.examples.build_example_strategy import build_example_strategy

strategy1 = build_example_strategy(lookback_period = 5, num_std = 1)
strategy2 = build_example_strategy(lookback_period = 10, num_std = 2)
portfolio = pq.Portfolio()
portfolio.add_strategy('bb_strategy_1', strategy1)
portfolio.add_strategy('bb_strategy_2', strategy2)
portfolio.run()

Lets now plot the output of the two strategies as well as the portfolio output.  The portfolio returns will be the sum of the returns of the 2 strategies.

In [ ]:
strategy1.plot(title = 'Strategy 1')

In [ ]:
strategy2.plot(title = 'Strategy 2')

In [ ]:
portfolio.plot()

In [ ]:
pq.__file__